In [ ]:
import pandas as pd

In [ ]:
file = r'..\dsa\raw\dataset_treino.csv'

df = pd.read_csv(file)

In [ ]:
df.head()

In [21]:
import sys
sys.path.append(r'C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/dsa_single_model')

import pandas as pd
import yaml
import pickle
import os
import json

# Carregando as configurações do arquivo YAML
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\dsa_single_model\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)   
    

    
if __name__ == "__main__":
    
      params = {
          'test_data': os.path.join(config['processed_data']['path'], 
                                    config['processed_data']['test']),
          'cols_2_drop_feat_sel' : config['feat_selection_params']['cols_2_drop'],           
          'pipe': config['pipe_feat_eng']['path'],
          'model': config['model']['path'],
          'model_version': config['model']['model_version'],
          'report': config['save_reports']['path_reports'],
          'target':config['feat_selection_params']['target'],
          'cols_2_drop_model':config['model_selection']['cols_2_drop'],
          'target':config['feat_selection_params']['target'],
          'submission':config['submission']['path']
          }
      
      
      

In [22]:
df_test = pd.read_parquet(params['test_data'])  

In [24]:
y_test_id = df_test[['id']].copy()

In [25]:
df_test.drop(
        columns=params['cols_2_drop_feat_sel'], 
        inplace=True)

In [26]:
# read pipe
pipe_path = os.path.join(
    params['model'],
    f"feat_sel_pipe_{config['feat_selection_params']['pipe_version']}.pkl")

with open(pipe_path, "rb") as file:
    pipe = pickle.load(file)

In [27]:
# appply pipe
df_test_transf = pipe.transform(df_test) 

df_test_transf.drop(
    columns = params['cols_2_drop_model'],
    inplace=True
)   

In [28]:
model_path = os.path.join(
    params['model'],
    f"model_{params['model_version']}.pkl")
# open model
with open(model_path, "rb") as file:
    model = pickle.load(file)

In [30]:
y_test_id.loc[:,f'{params['target'][0]}'] = model.predict(df_test_transf)

In [32]:
y_test_id.to_csv(
        os.path.join(params['submission'],
                     'submission.csv'),
        index=False)